## Volatility
### Microstructure noise determination

In [1]:
#Set up dependencies
import pandas as pd
import numpy as np
import io
import requests
from datetime import datetime, timedelta

In [2]:
spx_df = pd.read_csv("Cleaned_SPX_Data.csv")
spx_df = spx_df.drop(columns=['Unnamed: 0'])
tweet_df = pd.read_csv("mergedata_%change.csv")
tweet_df = tweet_df.drop(columns=['Unnamed: 0','Source'])
tweet_df = tweet_df.rename(columns={"Date Ti": "Date T_0", "SPX Ti": "SPX T_0", "Date Tf": "Date T_5", "SPX Tf": "SPX T_5"}) 

In [3]:
spx_df.head()

,Date,Close
0,20170102 1801,2243.00
1,20170102 1802,2243.25
2,20170102 1803,2243.00
3,20170102 1804,2243.75
4,20170102 1805,2243.75


In [4]:
tweet_df.head()
len(tweet_df)

2955

In [5]:
np.log(42/52)

-0.21357410029805904

In [44]:
#Create arrays of time 5 prior to +6 minutes after Tweet
neg_time = [-5, -4, -3, -2, -1, 0]
pos_time = [1, 2, 3, 4, 5, 6]

t_list_sub_5 = []
t_list_sub_4 = []
t_list_sub_3 = []
t_list_sub_2 = []
t_list_sub_1 = []
t_list_sub_0 = []
t_list_plus_1 = []
t_list_plus_2 = []
t_list_plus_3 = []
t_list_plus_4 = []
t_list_plus_5 = []
t_list_plus_6 = []


for row, index in tweet_df.iterrows():
    t_sub_5 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= -5)).strftime('%Y%m%d %H%M')
    t_list_sub_5.append(t_sub_5)
    t_sub_4 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= -4)).strftime('%Y%m%d %H%M')
    t_list_sub_4.append(t_sub_4)
    t_sub_3 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= -3)).strftime('%Y%m%d %H%M')
    t_list_sub_3.append(t_sub_3)
    t_sub_2 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= -2)).strftime('%Y%m%d %H%M')
    t_list_sub_2.append(t_sub_2)
    t_sub_1 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= -1)).strftime('%Y%m%d %H%M')
    t_list_sub_1.append(t_sub_1)
    t_sub_0 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M')).strftime('%Y%m%d %H%M')
    t_list_sub_0.append(t_sub_0)
               
    t_plus_1 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 1)).strftime('%Y%m%d %H%M')
    t_list_plus_1.append(t_plus_1)
    t_plus_2 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 2)).strftime('%Y%m%d %H%M')
    t_list_plus_2.append(t_plus_2) 
    t_plus_3 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 3)).strftime('%Y%m%d %H%M')
    t_list_plus_3.append(t_plus_3)
    t_plus_4 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 4)).strftime('%Y%m%d %H%M')
    t_list_plus_4.append(t_plus_4)
    t_plus_5 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 5)).strftime('%Y%m%d %H%M')
    t_list_plus_5.append(t_plus_5)
    t_plus_6 = (datetime.strptime(index['Date T_0'], '%Y%m%d %H%M') + timedelta(minutes= 6)).strftime('%Y%m%d %H%M')
    t_list_plus_6.append(t_plus_6) 
        
print(len(t_list_sub_5))

2955


In [43]:
t_list_sub_2

[]

In [46]:
# vol_sub_54_list = []
vol_sub_43_list = []
vol_sub_32_list = []
vol_sub_21_list = []
vol_sub_10_list = []
vol_sub_01_list = []
vol_plus_01_list = []
vol_plus_12_list = []
vol_plus_23_list = []
vol_plus_34_list = []
vol_plus_45_list = []
vol_plus_56_list = []


# for t_sub_5, t_sub_4 in zip(t_list_sub_5, t_list_sub_4):
#     try:
#         spx_sub_5 = spx_df.loc[spx_df['Date'] == t_sub_5, 'Close'].iloc[0]
#         spx_sub_4 = spx_df.loc[spx_df['Date'] == t_sub_4, 'Close'].iloc[0]
#         vol_sub_54 = np.log(spx_sub_4 / spx_sub_5)
#         print(vol_sub_54)
#         vol_sub_54_list.append(vol_sub_54)
#     except IndexError:
#         vol_sub_54_list.append("No SPX data avail.")
#         print("No SPX data avail.")


for t_sub_4, t_sub_3 in zip(t_list_sub_4, t_list_sub_3):
    try:
        spx_sub_4 = spx_df.loc[spx_df['Date'] == t_sub_4, 'Close'].iloc[0]
        spx_sub_3 = spx_df.loc[spx_df['Date'] == t_sub_3, 'Close'].iloc[0]
        vol_sub_43 = np.log(spx_sub_3 / spx_sub_4)
        vol_sub_43_list.append(vol_sub_43)
    except IndexError:
        vol_sub_43_list.append("No SPX data avail.")
print(f'Vol List 2 of 11 Complete')

for t_sub_3, t_sub_2 in zip(t_list_sub_3, t_list_sub_2):
    try:
        spx_sub_3 = spx_df.loc[spx_df['Date'] == t_sub_3, 'Close'].iloc[0]
        spx_sub_2 = spx_df.loc[spx_df['Date'] == t_sub_2, 'Close'].iloc[0]
        vol_sub_32 = np.log(spx_sub_2 / spx_sub_3)
        vol_sub_32_list.append(vol_sub_32)
    except IndexError:
        vol_sub_32_list.append("No SPX data avail.")
print(f'Vol List 3 of 11 Complete')

for t_sub_2, t_sub_1 in zip(t_list_sub_2, t_list_sub_1):
    try:
        spx_sub_2 = spx_df.loc[spx_df['Date'] == t_sub_2, 'Close'].iloc[0]
        spx_sub_1 = spx_df.loc[spx_df['Date'] == t_sub_1, 'Close'].iloc[0]
        vol_sub_21 = np.log(spx_sub_1 / spx_sub_2)
        vol_sub_21_list.append(vol_sub_21)
    except IndexError:
        vol_sub_21_list.append("No SPX data avail.")
print(f'Vol List 4 of 11 Complete')

for t_sub_1, t_sub_0 in zip(t_list_sub_1, t_list_sub_0):
    try:
        spx_sub_1 = spx_df.loc[spx_df['Date'] == t_sub_1, 'Close'].iloc[0]
        spx_sub_0 = spx_df.loc[spx_df['Date'] == t_sub_0, 'Close'].iloc[0]
        vol_sub_10 = np.log(spx_sub_0 / spx_sub_1)
        vol_sub_10_list.append(vol_sub_10)
    except IndexError:
        vol_sub_10_list.append("No SPX data avail.")
print(f'Vol List 5 of 11 Complete')

for t_sub_0, t_plus_1 in zip(t_list_sub_0, t_list_plus_1):
    try:
        spx_sub_0 = spx_df.loc[spx_df['Date'] == t_sub_0, 'Close'].iloc[0]
        spx_plus_1 = spx_df.loc[spx_df['Date'] == t_plus_1, 'Close'].iloc[0]
        vol_plus_01 = np.log(spx_plus_1 / spx_sub_0)
        vol_plus_01_list.append(vol_plus_01)
    except IndexError:
        vol_plus_01_list.append("No SPX data avail.")
print(f'Vol List 6 of 11 Complete')

for t_plus_1, t_plus_2 in zip(t_list_plus_1, t_list_plus_2):
    try:
        spx_plus_1 = spx_df.loc[spx_df['Date'] == t_plus_1, 'Close'].iloc[0]
        spx_plus_2 = spx_df.loc[spx_df['Date'] == t_plus_2, 'Close'].iloc[0]
        vol_plus_12 = np.log(spx_plus_2 / spx_plus_1)
        vol_plus_12_list.append(vol_plus_12)
    except IndexError:
        vol_plus_12_list.append("No SPX data avail.")
print(f'Vol List 7 of 11 Complete')

for t_plus_2, t_plus_3 in zip(t_list_plus_2, t_list_plus_3):
    try:
        spx_plus_2 = spx_df.loc[spx_df['Date'] == t_plus_2, 'Close'].iloc[0]
        spx_plus_3 = spx_df.loc[spx_df['Date'] == t_plus_3, 'Close'].iloc[0]
        vol_plus_23 = np.log(spx_plus_3 / spx_plus_2)
        vol_plus_23_list.append(vol_plus_23)
    except IndexError:
        vol_plus_23_list.append("No SPX data avail.")
print(f'Vol List 8 of 11 Complete')

for t_plus_3, t_plus_4 in zip(t_list_plus_3, t_list_plus_4):
    try:
        spx_plus_3 = spx_df.loc[spx_df['Date'] == t_plus_3, 'Close'].iloc[0]
        spx_plus_4 = spx_df.loc[spx_df['Date'] == t_plus_4, 'Close'].iloc[0]
        vol_plus_34 = np.log(spx_plus_4 / spx_plus_3)
        vol_plus_34_list.append(vol_plus_34)
    except IndexError:
        vol_plus_34_list.append("No SPX data avail.")
print(f'Vol List 9 of 11 Complete')

for t_plus_4, t_plus_5 in zip(t_list_plus_4, t_list_plus_5):
    try:
        spx_plus_4 = spx_df.loc[spx_df['Date'] == t_plus_4, 'Close'].iloc[0]
        spx_plus_5 = spx_df.loc[spx_df['Date'] == t_plus_5, 'Close'].iloc[0]
        vol_plus_45 = np.log(spx_plus_5 / spx_plus_4)
        vol_plus_45_list.append(vol_plus_45)
    except IndexError:
        vol_plus_45_list.append("No SPX data avail.")
print(f'Vol List 10 of 11 Complete')

for t_plus_5, t_plus_6 in zip(t_list_plus_5, t_list_plus_6):
    try:
        spx_plus_5 = spx_df.loc[spx_df['Date'] == t_plus_5, 'Close'].iloc[0]
        spx_plus_6 = spx_df.loc[spx_df['Date'] == t_plus_6, 'Close'].iloc[0]
        vol_plus_56 = np.log(spx_plus_6 / spx_plus_5)
        vol_plus_56_list.append(vol_plus_56)
    except IndexError:
        vol_plus_56_list.append("No SPX data avail.")
print('Vol List 11 of 11 Complete')

Vol List 2 of 11 Complete
Vol List 3 of 11 Complete
Vol List 4 of 11 Complete
Vol List 5 of 11 Complete
Vol List 6 of 11 Complete
Vol List 7 of 11 Complete
Vol List 8 of 11 Complete
Vol List 9 of 11 Complete
Vol List 10 of 11 Complete
Vol List 11 of 11 Complete


##### spx_sub_5 = spx_df.loc[spx_df['Date'] == t_sub_5, 'Close'].iloc[0]

print(t_sub_5)
print(spx_sub_5)

spx_sub_5 = spx_df.loc[spx_df['Date'] == t_sub_5, 'Close'].iloc[0]
spx_sub_5

In [52]:
print({"Vol -5 Mins Before Tweet" : vol_sub_54_list[0:4],
       "Vol -4 Mins Before Tweet" : vol_sub_43_list[0:4],
       "Vol -3 Mins Before Tweet" : vol_sub_32_list[0:4],
       "Vol -2 Mins Before Tweet" : vol_sub_21_list[0:4],
       "Vol -1 Mins Before Tweet" : vol_sub_10_list[0:4],
       "Vol @ Tweet" : vol_sub_01_list[0:4],
       "Vol 1 Mins After Tweet" : vol_plus_01_list[0:4],
       "Vol 2 Mins After Tweet" : vol_plus_12_list[0:4],
       "Vol 3 Mins After Tweet" : vol_plus_23_list[0:4],
       "Vol 4 Mins After Tweet" : vol_plus_34_list[0:4], 
       "Vol 5 Mins After Tweet" : vol_plus_45_list[0:4],
       "Vol 6 Mins After Tweet" : vol_plus_56_list[0:4]})

{'Vol -5 Mins Before Tweet': [0.0, -0.00011143923787093885, -0.00011156356345589146, 0.0], 'Vol -4 Mins Before Tweet': ['No SPX data avail.', 0.0, -0.00011157601127339085, 0.0003346533580231153], 'Vol -3 Mins Before Tweet': ['No SPX data avail.', -0.000111451657958756, -0.00011158846186880009, 0.0003345414026183912], 'Vol -2 Mins Before Tweet': ['No SPX data avail.', -0.00011146408081537711, -0.00011160091524334063, -0.00044608007876984136], 'Vol -1 Mins Before Tweet': [-0.00011140199421087216, 0.0, 0.00033476538838554703, -0.00011155111841563559], 'Vol @ Tweet': [], 'Vol 1 Mins After Tweet': ['No SPX data avail.', 0.0, 0.0, 0.00022308979456711704], 'Vol 2 Mins After Tweet': ['No SPX data avail.', 0.0, 0.0003346533580231153, -0.00011153867615151279], 'Vol 3 Mins After Tweet': ['No SPX data avail.', 0.000445781797196129, 0.0003345414026183912, 0.00011153867615155138], 'Vol 4 Mins After Tweet': ['No SPX data avail.', -0.00011142682055092624, -0.00044608007876984136, 0.0004460303374570626

In [26]:
vol_sub_5

    A  B
0  p1  1
1  p1  2
2  p3  3
3  p2  4
df.loc[df['B'] == 3, 'A'].iloc[0]

'20181230 2159'

In [38]:
from datetime import datetime

datetime_test = datetime.strptime('20181230 2159', '%Y%m%d %H%M')
datetime_test  - timedelta(minutes=  5)

datetime.datetime(2018, 12, 30, 21, 54)